<a href="https://colab.research.google.com/github/datarobot-community/DRU-MLOps/blob/master/24Feb2022 - DRUM_Lab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Custom Model Inegration with MLOps - DRUM Laboratory

Welcome to the MLOps III DRUM Lab! 
 
 In this notebook we will

* Build a simple regression model using Scikit-Learn
* Use DRUM to test & validate the model
* Use DRUM to score data in batch mode


## Use case to be addressed:

We will build a regression model to predict depression scores in older adults. 

Let's begin by uploading a few resources we will need:

1. Training set: **mental_health_training.csv**
2. Scoring set: **mental_health_inference.csv**
3. Requirements file: **colab_requirements.txt**
4. File with hooks used by the model: **custom.py**

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
!ls

We will now create parameters to pass the names of the training and inference datasets to the next cells. First we will create a parameter for the name of the training dataset:

In [ ]:
TRAINING = 'mental_health_training.csv'

We will define an environment variable to pass the name of the inference dataset to the DRUM command:

In [ ]:
INFERENCE = 'mental_health_inference.csv'

In [ ]:
!export INFERENCE

Let's install the Python modules we need using the requirements file:

In [ ]:
!pip install -r colab_requirements.txt -q

In [ ]:
import os
os.environ['PYTHONWARNINGS'] = 'ignore'

# 1.- Model Training

We will now build a very simple Scikit-Learn Regression model using the boston_housing prices dataset.

In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
import pickle
import datetime

## load data
df = pd.read_csv(TRAINING)
df.head()

In [ ]:
## set features and target
X = df.drop('depression_score', axis=1)
y = df['depression_score']

## train the model
rf = RandomForestRegressor(n_estimators = 20, random_state = 801)
rf.fit(X,y)

## serialize the model
with open('rf.pkl', 'wb') as pkl:
    pickle.dump(rf, pkl)

print("Done!")    

# 2.- Model Testing

We will now use DRUM to test how the model performs by computing latency times and memory usage for several different test case sizes. A report is generated after this process is completed.



In [ ]:
!drum perf-test 

# 3.- Model Validation: Handling of Missing Values

We will now validate the model to detect and address issues before deployment. It’s highly encouraged that you run these tests, which are the same ones that DataRobot performs automatically before deploying models.

Especifically, DRUM will test null values imputation by setting each feature in the dataset to "missing" and then feeding the features to the model. We will send the results to **validation.log**

In [ ]:
!drum validation 

In [ ]:
!cat validation.log

# 4.- Batch Scoring with DRUM
<a id="setup_complete"></a>

We want to use our model to make predictions; to do this, we'll leverage DRUM and its ability to natively handle our Scikit-Learn model. All we need to do is tell DRUM where the model resides and what data we wish to score.  

DRUM provides native support for many frameworks. To use DRUM with model frameworks that are not supported out-of-the box, we'll just need to create some custom hooks so DRUM.  In this example, we'll explain some very simple custom hooks and provide links to more complex examples.  

In [ ]:
!drum score 

Let's have a look at the predictions:

In [ ]:
!cat predictions.csv